# Support Vector Machine
### Nataly Neira 614212782
### Sebastian Mantilla 614221025

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statistics as sts
import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix,mean_squared_error, r2_score

In [ ]:
data = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Crime_Data_from_2020_to_Present.csv'

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
missing_v = pd.DataFrame(data.isna().sum())
missing_v = missing_v.rename(columns = {0: 'sum_missing_values'})
missing_v['porcentaje'] = (missing_v['sum_missing_values']/len(data))*100
missing_v

In [ ]:
columns_to_drop = missing_v[missing_v['porcentaje'] >= 10].index
data = data.drop(columns=columns_to_drop)
data.isna().sum()

Se han seleccionado y eliminado las columnas que tienen más del 10% de datos faltantes.
Para efectos de este modelo, se eliminarán [LOCATION], [LAT], [LON] que tienen la misma información del Área
Se eliminan también los datos faltantes de las columnas Premis Cd, Premis Desc y Crm Cd 1 para no afectar el modelo.
Se elimina también Date Rptd y Crm Cd 1 ya que no aportan información adicional al modelo al igual que las descripciones de Premis y Status y Crime Code.

De igual manera se eliminan los datos en los que la edad es cero para no afectar el desempeño del modelo ya que estos datos que son erroneos pueden introducir un sesgo.

In [ ]:
data = data.dropna()
data = data.drop(columns=['Date Rptd','DR_NO','Premis Desc','Status Desc','LOCATION','LAT','LON', 'Crm Cd 1', 'Crm Cd Desc', 'AREA NAME'])
data = data[data['Vict Age'] > 0]
data.isna().sum()

In [ ]:
data.info()

Ahora vamos a clasificar las edades de la victima en diferentes buckets para tratar mejor los datos categóricos.

# Tratamiento de variables categóricas

Con las 10 columnas que nos quedan, tenemos 2 columnas categóricas que no son de tipo numérico. Vamos transformar los datos de estas columnas en datos numéricos.

In [ ]:
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p').astype(int)

Primero lo hacemos con la columna Date Occ.
Ahora lo haremos con la columna Status.

In [ ]:
status_cont=data.groupby(['Status']).size().reset_index(name='conteo')
status_cont

In [ ]:
#Mapeo Status
mapeo_status={'AA':0,'AO':1,'CC':2,'IC':3,'JA':4,'JO':5}
data['Status']=data['Status'].map(mapeo_status)

In [ ]:
data.head()

Vamos a clasificar las edades en buckets

In [ ]:
def map_age_group(age):
    if age <= 18:
        return 1
    else:
        return 2

data['Vict Age'] = data['Vict Age'].apply(map_age_group)

In [ ]:
data.info()

# Clasificación con SVM

In [ ]:
# Separamos en Variables independientes y objetivo
X = data.drop(columns=['Vict Age'])
y = data['Vict Age']

In [ ]:
#rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

#rfe = RFE(rf_classifier, n_features_to_select=2)
#X_rfe = rfe.fit_transform(X, y)

#selected_features = X.columns[rfe.support_]
#selected_features

In [ ]:
## Separamos de nuevo las variables reduciendo el set X
X = data[['Rpt Dist No']]
y = data['Vict Age']
#Separamos nuestros datasets de train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Escalamos todos los features para evitar sesgos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Entrenamos el modelo
svm_classifier = SVC(kernel='poly', degree=2, C=1E3)
svm_classifier.fit(X_train_scaled, y_train)

# Predecimos los demás valores
y_pred = svm_classifier.predict(X_test_scaled)

In [ ]:
#Revisamos las métricas:
report = classification_report(y_test, y_pred)
print(report)